In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

In [ ]:
from filters import *
from masking import *
from simulCAP import *
from latencies import *
from excitation import *
from tests import *
from deconv import *

### Parameters for every signals

In [ ]:
t=ConvolutionCAPSimulatorSingleFilterModel.default_t_array()
md=SigmoidMaskingDegreeFunction(30, 2*1/15., 0.95)

plotMaskingDegreeFunc(md)

### True parameters (reference)

In [ ]:
lat0=Eggermont1976clickLatencies80dB #latencies

#Unitary response
ur0=URWang1979m
u00=ur0.u(t-2e-3) #shift
from scipy.ndimage  import gaussian_filter1d
sig=0.3e-3 #std of gaussian kernel in s
sig2=sig/(t[1]-t[0])
u0 = gaussian_filter1d(u00, sigma=sig2)
u0/=np.amax(np.abs(u0))
u0*=0.5*np.amax(np.abs(u00))

f_c = 3000
Q_10_0 = 4 #at f_c

#Excitation pattern
EPref=EP0= ExcitationPattern(lat0, f_c, 1.7, 70)

gf0= GaussianFilter.givenQ10(f_c, Q_10_0)


Latencies

In [ ]:
plotLatencies(lat0)

Unitary response

In [ ]:
pl.plot(t*1e3, u00, label='unitary response')
pl.plot(t*1e3, u0, label='ur + nPSHT (blur)',  color='orange')
pl.legend(loc='lower right')
pl.title("Unitary response")
pl.xlabel('t (ms)')

Example : excitation pattern with tone masker

In [ ]:
A=10**(45/20.)*np.sqrt(gf0.ERB())

mask=ToneSingleFilterMaskingPattern(A, f_c*0.9, gf0, md) 
EPm = ExcitationPattern.mask(EP0, mask)
with pl.style.context('default'):
    plotExcitationPattern(EPm, figsize=(8, 3))
    plotSimulCAPs2convGaussianKernel([EPm], sig=sig, secondPeak=0., figsize=(9, 7), plotGaussianKernel=False, plotmaskingpattern=False)


### Different masking settings

In [ ]:
nb_tones=8 #nb tone maskers
nb_noise=nb_tones #nb high pass noise maskers

freqs=np.linspace(0.7, 1.1, nb_tones)*f_c

#Tones
A=10**(45/20.)*np.sqrt(gf0.ERB())
#some variations
A_arr=A*10**(2*np.random.randn((nb_tones))/10.)
mc_tones = [ToneSingleFilterMaskingCondition(A_arr[i], freqs[i]) for i in range(nb_tones)]

#HP noise
Ihz=55+3*np.random.randn((nb_noise))
mc_hpnoises = [HighPassNoiseSingleFilterMaskingCondition(Ihz[i], freqs[i]) for i in range(nb_noise)]

mcs = [NoMaskingCondition()]+mc_tones+mc_hpnoises

capSimulator = ConvolutionCAPSimulatorSingleFilterModel(lat0, gf0, EP0, md, mcs, ur=u0)

plotConvCAPSimulator(capSimulator, figsize=(14,12))

### CAP signals

Add noise

In [ ]:
CAPs0=capSimulator.simulCAPs()
CAPamax=np.amax(np.abs(CAPs0))
CAPs=CAPs0+1/15*CAPamax*np.random.randn(*np.shape(CAPs0))

In [ ]:
pl.figure(figsize=(12, 5))
pl.suptitle('CAPs + noise')
m=capSimulator.m
nb_col=(m+1)//2 if m<=12 else (m+2)//3
nb_row=(m+nb_col-1)//nb_col
for i in range(m):
    ind=i+1
    pl.subplot(nb_row, nb_col, ind)
    pl.plot(t*1e3, CAPs[i])
    pl.xlabel('t (ms)')
    pl.ylim([-1.2*CAPamax, 1.2*CAPamax])
    pl.gca().get_yaxis().set_visible(False)
pl.show()

Wiener filter + low pass

In [ ]:
CAPs_f=np.zeros_like(CAPs)
window=sg.tukey(np.shape(CAPs)[1], alpha=0.2)
for i in range(capSimulator.m):
    #window signal (tuckey window)
    CAP_w=window*CAPs[i]
    CAPs_f[i]=sg.wiener(CAP_w)
    #low pass (gaussian kernel to simply for now)
    sig_f=0.15e-3 #in s
    sig_f=sig_f/(t[1]-t[0])
    CAPs_f[i]=gaussian_filter1d(CAPs_f[i], sigma=sig_f)

In [ ]:
pl.figure(figsize=(12, 5))
pl.suptitle('CAPs (ref. and filtered)')
m=capSimulator.m
nb_col=(m+1)//2 if m<=12 else (m+2)//3
nb_row=(m+nb_col-1)//nb_col
for i in range(m):
    ind=i+1
    pl.subplot(nb_row, nb_col, ind)
    pl.plot(t*1e3, CAPs0[i])
    pl.plot(t*1e3, CAPs_f[i])
    pl.xlabel('t (ms)')
    pl.ylim([-1.2*CAPamax, 1.2*CAPamax])
    pl.gca().get_yaxis().set_visible(False)
pl.show()

In [ ]:
#deconvs
#TODO vectorize?

#given u
u_fft=np.fft.rfft(u0)
EP_deconv=np.zeros_like(CAPs_f)
for i in range(capSimulator.m):
    CAP_fft=np.fft.rfft(CAPs_f[i])
    EP_fft=CAP_fft/u_fft
    EP_deconv[i]=np.fft.irfft(EP_fft)
    
#given EPs
EPs0=capSimulator.getEPs()
u_deconv=np.zeros_like(CAPs_f)
for i in range(capSimulator.m):
    CAP_fft=np.fft.rfft(CAPs_f[i])
    EP_fft=np.fft.rfft(EPs0[i])
    u_fft=CAP_fft/EP_fft
    u_deconv[i]=np.fft.irfft(u_fft)
    

In [ ]:
def plotMatrices(X, X_ref, title=''):
    pl.figure(figsize=(12, 5))
    pl.suptitle(title)
    m=capSimulator.m
    nb_col=(m+1)//2 if m<=12 else (m+2)//3
    nb_row=(m+nb_col-1)//nb_col
    for i in range(m):
        ind=i+1
        pl.subplot(nb_row, nb_col, ind)
        pl.plot(t*1e3, X_ref[i])
        pl.plot(t*1e3, X[i])
        pl.xlabel('t (ms)')
        pl.gca().get_yaxis().set_visible(False)
    pl.show()

In [ ]:
EPs0=capSimulator.getEPs()
plotMatrices(EP_deconv, EPs0, title='Test deconv of EPs (if u is known)')

In [ ]:
u0_mat=np.tile(u0, (capSimulator.m, 1))
plotMatrices(u_deconv, u0_mat, title='Test deconv of u (if EPs are known)')

In [ ]:
# with gradient descent (/Newton algorithm) + projs
#given u
u_fft=np.fft.rfft(u0)

EPs0=capSimulator.getEPs()
EP1=ExcitationPattern(lat0, f_c, 1.5, 50)
EP_deconv=np.tile(EP1.E(t), (capSimulator.m, 1))
nb_steps=6
alpha=0.5
u_fft=np.fft.rfft(u0)
CAPs_fft=np.fft.rfft(CAPs_f, axis=1)
for i in range(1, nb_steps+1):
    dEP=deconv_newton_step(EP_deconv, u_fft, CAPs_fft)
    #print(f"step {i}, grad/EP: {grad_norm/EPs_norm:.2f}")
    EP_deconv-=alpha*dEP
    if i%2==0:
        pass
        #plotMatrices(EP_deconv, EPs0, title=f'Test deconv of EPs (u is known) w/ grad. descent step {i}')
    
    
#given EPs
nb_steps=6
alpha=0.3
#other EP
#capSimulator1 = ConvolutionCAPSimulatorSingleFilterModel(lat0, gf0, EP1, md, mcs, ur=u0)
#EPs1=capSimulator1.getEPs()
#EPs_fft=np.fft.rfft(EPs1, axis=1)
EPs_fft=np.fft.rfft(EPs0, axis=1)
u0_mat=np.tile(u0, (capSimulator.m, 1))
u1_mat=np.zeros_like(CAPs_f)
filter_mat = t>6e-3
filter_mat=np.tile(filter_mat, (capSimulator.m, 1))
for i in range(1, nb_steps+1):
    du=deconv_newton_step(u1_mat, EPs_fft, CAPs_fft)
    #print(f"step {i}, grad/EP: {grad_norm/EPs_norm:.2f}")
    u1_mat-=alpha*du
    #proj 1 
    u1_mat[filter_mat]=np.zeros_like(u1_mat[filter_mat])
    #proj 2
    u1_mat_mean=np.mean(u1_mat, axis=0)[None, :]
    u1_mat=np.repeat(u1_mat_mean, capSimulator.m, axis=0)
    if i%2==0:
        plotMatrices(u1_mat, u0_mat, title=f'Test deconv of u (EPs are known) w/ grad. descent step {i}')
    